In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python# For example, here's several helpful packages to loadimport numpy as np # linear algebraimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)# Input data files are available in the read-only "../input/" directory# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directoryimport osfor dirname, _, filenames in os.walk('/kaggle/input'):    for filename in filenames:        print(os.path.join(dirname, filename))# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install openpyxl # Install the openpyxl library using pip

In [4]:
import pandas as pd # Import the pandas library and assign it to the alias 'pd'
df = pd.read_excel('/content/glassdoor_reviews_COPY_FULL.xlsx')
# Now you can work with the dataframe 'df'
# Example: print the first 5 rows
df.head()

,Review ID,Company Name,Overall Rating,Header,Job Title,Employment Status,Employment Tenure,Location,Recommend,CEO Approval,Business Outlook,Pros,Cons,Review Date
0,1,Deloitte,3,Good learning,Senior manager,Current employee,more than 10 years,Mumbai,Yes,Yes,Yes,Lot of learning and to work on latest technolo...,Work life fit and bureaucracy,"Oct 28, 2024"
1,2,Deloitte,3,Work environment,Senior tax manager,Current employee,more than 10 years,Portugal,No,No Data,No,Acess to information and a lot of learning cou...,Environment very competitive between colleague...,"Oct 28, 2024"
2,3,Deloitte,4,Good work culture,Analyst,Current employee,less than 1 year,Hyderābād,Yes,Yes,Yes,Flexi and nice people communication,Less salary and also growth is slow,"Oct 28, 2024"
3,4,Deloitte,4,Good place,Senior developer,Current employee,NaN,Bengaluru,No Data,No Data,No Data,Good working culture in the office,Less salary and less increments,"Oct 28, 2024"
4,5,Deloitte,3,Life at deloitte,Tax consultant,Former employee,NaN,Pune,No Data,No Data,No Data,"Great working environment, Good mentoring",Too many working hours and no extra payment,"Oct 28, 2024"


In [5]:
# prompt: Remove stop words from Pros column and Cons column and change it all to lower case

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if isinstance(text, str):  # Check if the input is a string
        words = text.lower().split()
        filtered_words = [word for word in words if word not in stop_words]
        return " ".join(filtered_words)
    else:
        return text  # Return the original value if it's not a string

df['Pros'] = df['Pros'].apply(remove_stopwords)
df['Cons'] = df['Cons'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# prompt: Remove special characters from Pros and Cons column. Also remove the word "page" from it

import re

def remove_special_characters(text):
    if isinstance(text, str):
        # Remove special characters and the word "page"
        text = re.sub(r'[^\w\s]', '', text)
        text = text.replace("page", "")
        return text
    else:
        return text

df['Pros'] = df['Pros'].apply(remove_special_characters)
df['Cons'] = df['Cons'].apply(remove_special_characters)
df.head()

,Review ID,Company Name,Overall Rating,Header,Job Title,Employment Status,Employment Tenure,Location,Recommend,CEO Approval,Business Outlook,Pros,Cons,Review Date
0,1,Deloitte,3,Good learning,Senior manager,Current employee,more than 10 years,Mumbai,Yes,Yes,Yes,lot learning work latest technologies,work life fit bureaucracy,"Oct 28, 2024"
1,2,Deloitte,3,Work environment,Senior tax manager,Current employee,more than 10 years,Portugal,No,No Data,No,acess information lot learning courses,environment competitive colleagues,"Oct 28, 2024"
2,3,Deloitte,4,Good work culture,Analyst,Current employee,less than 1 year,Hyderābād,Yes,Yes,Yes,flexi nice people communication,less salary also growth slow,"Oct 28, 2024"
3,4,Deloitte,4,Good place,Senior developer,Current employee,NaN,Bengaluru,No Data,No Data,No Data,good working culture office,less salary less increments,"Oct 28, 2024"
4,5,Deloitte,3,Life at deloitte,Tax consultant,Former employee,NaN,Pune,No Data,No Data,No Data,great working environment good mentoring,many working hours extra payment,"Oct 28, 2024"


In [7]:
# prompt: lemmatize words in Pros and Cons columns so that they convert to base
!pip install nltk
import nltk

# Download the required data package
nltk.download('punkt_tab')

# ... (your existing code) ...

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def lemmatize_text(text):
    """Lemmatizes words in a given text."""
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)  # Tokenize the text into words
    lemmas = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmas)  # Join the lemmas back into a string

# Create the 'wordsdf' DataFrame by copying 'df'
wordsdf = df.copy()  # Now 'wordsdf' is defined

wordsdf['Pros'] = df['Pros'].astype(str).apply(lemmatize_text)
wordsdf['Cons'] = df['Cons'].astype(str).apply(lemmatize_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# prompt: create a new column "employment status_encoded" which has Employment status values 0 for Former employee and 1 for Current employee

# Create the new column 'employment status_encoded'
df['employment status_encoded'] = df['Employment Status'].map({'Former employee': 0, 'Current employee': 1})

In [9]:
# prompt: create a new column "Review" that joins the Pros and Cons columns

# Assuming 'df' is your DataFrame as defined in the previous code
df['Review'] = df['Pros'].astype(str) + ' ' + df['Cons'].astype(str)
df.head()

,Review ID,Company Name,Overall Rating,Header,Job Title,Employment Status,Employment Tenure,Location,Recommend,CEO Approval,Business Outlook,Pros,Cons,Review Date,employment status_encoded,Review
0,1,Deloitte,3,Good learning,Senior manager,Current employee,more than 10 years,Mumbai,Yes,Yes,Yes,lot learning work latest technologies,work life fit bureaucracy,"Oct 28, 2024",1,lot learning work latest technologies work lif...
1,2,Deloitte,3,Work environment,Senior tax manager,Current employee,more than 10 years,Portugal,No,No Data,No,acess information lot learning courses,environment competitive colleagues,"Oct 28, 2024",1,acess information lot learning courses environ...
2,3,Deloitte,4,Good work culture,Analyst,Current employee,less than 1 year,Hyderābād,Yes,Yes,Yes,flexi nice people communication,less salary also growth slow,"Oct 28, 2024",1,flexi nice people communication less salary al...
3,4,Deloitte,4,Good place,Senior developer,Current employee,NaN,Bengaluru,No Data,No Data,No Data,good working culture office,less salary less increments,"Oct 28, 2024",1,good working culture office less salary less i...
4,5,Deloitte,3,Life at deloitte,Tax consultant,Former employee,NaN,Pune,No Data,No Data,No Data,great working environment good mentoring,many working hours extra payment,"Oct 28, 2024",0,great working environment good mentoring many ...


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Sample data
texts = df['Review']
labels = df['employment status_encoded']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Define pipeline with optimizations
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.95, min_df=0.05,ngram_range=(1, 2))),
    ('lda', LatentDirichletAllocation(n_components=2, random_state=42, learning_method='online',max_iter=10, learning_decay=0.7, n_jobs=-1)),
    ('classifier', RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f" RandomForestClassifier Accuracy: {score}")


 RandomForestClassifier Accuracy: 0.6504576083676958


In [33]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.95, min_df=0.05,ngram_range=(1, 2))),
    ('lda', LatentDirichletAllocation(n_components=2, random_state=42, learning_method='online',max_iter=10, learning_decay=0.7, n_jobs=-1)),
    ('classifier', LogisticRegression(random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"LogisticRegression Accuracy: {score}")


LogisticRegression Accuracy: 0.6504073217338832


In [34]:
from sklearn.svm import SVC

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.95, min_df=0.05,ngram_range=(1, 2))),
    ('lda', LatentDirichletAllocation(n_components=2, random_state=42, learning_method='online',max_iter=10, learning_decay=0.7, n_jobs=-1)),
    ('classifier', SVC(kernel='linear', random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"SVC Accuracy: {score}")


SVC Accuracy: 0.6504073217338832


In [35]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.95, min_df=0.05,ngram_range=(1, 2))),
    ('lda', LatentDirichletAllocation(n_components=2, random_state=42, learning_method='online',max_iter=10, learning_decay=0.7, n_jobs=-1)),
    ('classifier', GradientBoostingClassifier(n_estimators=50, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"Accuracy: {score}")


Accuracy: 0.6505078950015086


In [36]:
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.95, min_df=0.05,ngram_range=(1, 2))),
    ('lda', LatentDirichletAllocation(n_components=2, random_state=42, learning_method='online',max_iter=10, learning_decay=0.7, n_jobs=-1)),
    ('classifier', MultinomialNB())
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"MultinomialNB Accuracy: {score}")


MultinomialNB Accuracy: 0.6504073217338832
